In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from store import *
import kbech32

In [2]:
store = Store('/home/jupyter/datadir2')

In [3]:
store.load_recent_blocks(7200_000)

In [4]:
header_fields = ['timeInMilliseconds', 'blueScore', 'blueWork', 'daaScore', 'difficulty']
block_fields = ['pubkey_script', 'kaspad_version', 'miner_version']
count_fields = [] # ['num_parents', 'num_children', 'num_blues', 'num_reds']
frames = store.load_data(header_fields=header_fields, block_fields=block_fields, count_fields=count_fields)

  0%|          | 0/7132 [00:00<?, ?it/s]

In [5]:
frames.keys()

dict_keys(['hash', 'timeInMilliseconds', 'blueScore', 'blueWork', 'daaScore', 'difficulty', 'pubkey_script', 'kaspad_version', 'miner_version'])

In [6]:
df = pd.DataFrame(frames).set_index('hash')

In [7]:
pp_time = store.get_header_data(store.pruning_point()).timeInMilliseconds
pp_time, len(df)

(1703170010165, 7132)

In [8]:
df['kaspad_version'].value_counts(normalize=True)

kaspad_version
0.12.15    0.994952
0.12.14    0.005048
Name: proportion, dtype: float64

In [9]:
def miner_type(miner_version):
    if 'BzMiner' in miner_version:
        return 'BzMiner'
    if '0.2.1-GPU' in miner_version:
        return 'CommunityMiner'
#     if 'WoolyPooly' in miner_version:
#         return 'WoolyPooly'
#     if 'acc-pool' in miner_version:
#         return 'acc-pool'
    if 'KStratum' in miner_version:
        return 'KStratum'
    if 'onemorebsmith' in miner_version:
        return 'onemorebsmith-bridge'
    return miner_version

df["miner_type"] = df["miner_version"].apply(miner_type)

In [12]:
version_and_miner_type = (df['kaspad_version'] + '/' + df['miner_type']).value_counts(normalize=True)

In [13]:
version_and_miner_type

0.12.15/kaspa-pool.org                      0.122406
0.12.15/96380d695ec82f5b884dfe54386af091    0.082586
0.12.15/5b376cc2acaf36edfb04162b588b3ecf    0.055805
0.12.15/f478263d17acd1f7288d738689c05aa5    0.045990
0.12.15/onemorebsmith-bridge                0.044448
                                              ...   
0.12.14/eu2.acc-pool.pw                     0.000140
0.12.15/gpumine.org agent-pro-002/f5ab      0.000140
0.12.15/Mined by in.kaspa.herominers.com    0.000140
0.12.15/gpumine.org agent-pro-002/734b      0.000140
0.12.15/gpumine.org agent-pro-002/da37      0.000140
Name: proportion, Length: 233, dtype: float64

In [14]:
miner_type = df['miner_type'].value_counts(normalize=True)

In [15]:
miner_type

miner_type
kaspa-pool.org                      0.122406
96380d695ec82f5b884dfe54386af091    0.082586
5b376cc2acaf36edfb04162b588b3ecf    0.055805
onemorebsmith-bridge                0.047112
f478263d17acd1f7288d738689c05aa5    0.045990
                                      ...   
gpumine.org agent-pro-002/f5ab      0.000140
Mined by in.kaspa.herominers.com    0.000140
gpumine.org agent-pro-002/734b      0.000140
2miners-eu-02                       0.000140
gpumine.org agent-pro-002/da37      0.000140
Name: proportion, Length: 230, dtype: float64

In [25]:
for entry in version_and_miner_type.items(): #Full list of versions and miner types
	print(entry[0].ljust(45), round(entry[1],4))

0.12.15/kaspa-pool.org                        0.1224
0.12.15/96380d695ec82f5b884dfe54386af091      0.0826
0.12.15/5b376cc2acaf36edfb04162b588b3ecf      0.0558
0.12.15/f478263d17acd1f7288d738689c05aa5      0.046
0.12.15/onemorebsmith-bridge                  0.0444
0.12.15/134e77530b7bc0f13dc19bc17fff4999      0.0435
0.12.15/kaspaminer-0.12.13 ap86               0.0353
0.12.15/kaspaminer-0.12.13 ap83               0.0315
0.12.15/kaspaminer-0.12.13 ap85               0.0301
0.12.15/kaspaminer-0.12.13 ap84               0.0297
0.12.15/kaspaminer-0.12.13 ap82               0.0271
0.12.15/tw-pool1-0                            0.0261
0.12.15/tw-pool2-0                            0.0167
0.12.15/4d9f2b504202355616e9fad02283df36      0.0139
0.12.15/kaspaminer-0.12.13                    0.0129
0.12.15/ANTPOOL-HK1                           0.0126
0.12.15/ANTPOOL-HK2                           0.0126
0.12.15/3fd6bdd3d5caeaabd8a1cd030c71f7b2      0.0125
0.12.15/bc9a3d7eb2c90540c0d837cd3c8948fe      0

In [ ]:
for entry in version_and_miner_type.items(): #Full list of versions and miner types
	print(entry[0].ljust(45), round(entry[1],4))

In [12]:
store.close()